## import all the required files and libraries

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import copy
import os
import numpy as np
from numpy import genfromtxt
import tensorflow as tf

import tensorflow as tf
import numpy as np
import os
import cv2
from numpy import genfromtxt
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
import h5py

Using TensorFlow backend.


#### import the necessary modules (i.e. weights,models etc.)

In [4]:
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

create the model

In [5]:
#np.set_printoptions(threshold=np.nan)
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

# triplet loss

In [6]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    ### START CODE HERE ### (≈ 4 lines)
# Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)))
# Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)))
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = pos_dist-neg_dist+alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.math.maximum(basic_loss,0))
    ### END CODE HERE ###
    
    return loss

#### loading the trained model

In [7]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

#### read the 1st image

In [18]:
image1="_dataset/400.jpg"         # _dataset folder contain cropped faces of students
#image1="taken_dataset/400.jpg"
image = cv2.imread(image1)
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
row,col,chan = image.shape
# resize the original image as per requirement 

reimage=cv2.resize(image,(96,96),interpolation =cv2.INTER_AREA )
for n in range (3):
    # initialze an array color required for padding
    coll = image[0, 0, n]   #coll = gray[row - 1, col - 1]
    color = [int(coll), int(coll), int(coll)]
    
    if row>col:
        teimage =reimage[:,:,n]
        row, col = teimage.shape
        left = right = ((96 - col)/2)
        sqimage = cv2.copyMakeBorder(teimage, 0, 0, int(left), int(right), cv2.BORDER_CONSTANT, value=color)
        row, col = sqimage.shape
        if(row != col):
            sqimage = cv2.copyMakeBorder(sqimage, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value=color)
       # overwrite the subsequent channel of original image
        reimage[:,:,n]=sqimage
        
    else:
       # reimage = image_resize(image[:,:,n], width=96)
        teimage=reimage[:,:,n]
        row, col = teimage.shape
        top = bottom = (96 - row)/2
        sqimage = cv2.copyMakeBorder(teimage, int(top), int(bottom), 0, 0, cv2.BORDER_CONSTANT, value=color)
        row, col = sqimage.shape
        if (row != col):
            sqimage = cv2.copyMakeBorder(sqimage, 0, 1, 0, 0, cv2.BORDER_CONSTANT, value=color)
     # overwrite the subsequent channel of original image
        reimage[:,:,n]=sqimage
# save the processed image in the system
cv2.imwrite("C:/Users/Lenovo/Deep_learning/TanuJ_sir/padded.jpg", reimage)
# create the encoding of 1st image using siamese network
arr = img_to_encoding("C:/Users/Lenovo/Deep_learning/TanuJ_sir/padded.jpg", FRmodel)

### read the second image


In [9]:

image2="eg/10.jpg"         #in eg folder I store images of different persons

immage = cv2.imread(image2)
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
row,col,chan = immage.shape

reimage2=cv2.resize(immage,(96,96),interpolation =cv2.INTER_AREA)   
for n in range (3):
    # initialze an array color required for padding
    coll = immage[0, 0, n]   #coll = gray[row - 1, col - 1]
    color = [int(coll), int(coll), int(coll)]
    
    if row>col:
        teimage2 =reimage2[:,:,n]
        row, col = teimage2.shape
        left = right = ((96 - col)/2)
        sqimage = cv2.copyMakeBorder(teimage2, 0, 0, int(left), int(right), cv2.BORDER_CONSTANT, value=color)
        row, col = sqimage.shape
        if(row != col):
            sqimage = cv2.copyMakeBorder(sqimage, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value=color)
       # overwrite the subsequent channel of original image
        reimage2[:,:,n]=sqimage
        
    else:
       # reimage = image_resize(image[:,:,n], width=96)
        teimage2=reimage2[:,:,n]
        row, col = teimage2.shape
        top = bottom = (96 - row)/2
        sqimage = cv2.copyMakeBorder(teimage2, int(top), int(bottom), 0, 0, cv2.BORDER_CONSTANT, value=color)
        row, col = sqimage.shape
        if (row != col):
            sqimage = cv2.copyMakeBorder(sqimage, 0, 1, 0, 0, cv2.BORDER_CONSTANT, value=color)
            #k
     # overwrite the subsequent channel of original image
        reimage2[:,:,n]=sqimage
# save the processed image in the system
cv2.imwrite("C:/Users/Lenovo/Deep_learning/TanuJ_sir/padded2.jpg",reimage2)
arr2 = img_to_encoding("C:/Users/Lenovo/Deep_learning/TanuJ_sir/padded2.jpg", FRmodel)
arr2.shape

(1, 128)

#### find the distance between two different images

In [10]:
diff=np.linalg.norm((arr-arr2))
print(diff)

1.0466303



#### run this blocks to save padded images from dataset inthe directory

In [12]:
# initialize counter to count total no of images inthe dataset
count=0

for i in os.listdir('_dataset'): 
    count=count+1
    #image1=i
    image = cv2.imread(os.path.join('_dataset',i))
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #reimage=padding_image(image)
    row,col,chan = image.shape
    # resize the original image as per requirement 


    reimage=cv2.resize(image,(96,96),interpolation =cv2.INTER_AREA )
    for n in range (3):
        # initialze an array color required for padding
        coll = image[0, 0, n]   #coll = gray[row - 1, col - 1]
        color = [int(coll), int(coll), int(coll)]
    
        if row>col:
            teimage =reimage[:,:,n]
            row, col = teimage.shape
            left = right = ((96 - col)/2)
            sqimage = cv2.copyMakeBorder(teimage, 0, 0, int(left), int(right), cv2.BORDER_CONSTANT, value=color)
            row, col = sqimage.shape
            if(row != col):
                sqimage = cv2.copyMakeBorder(sqimage, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value=color)
       # overwrite the subsequent channel of original image
            reimage[:,:,n]=sqimage
        
        else:
       # reimage = image_resize(image[:,:,n], width=96)
            teimage=reimage[:,:,n]
            row, col = teimage.shape
            top = bottom = (96 - row)/2
            sqimage = cv2.copyMakeBorder(teimage, int(top), int(bottom), 0, 0, cv2.BORDER_CONSTANT, value=color)
            row, col = sqimage.shape
            if (row != col):
                sqimage = cv2.copyMakeBorder(sqimage, 0, 1, 0, 0, cv2.BORDER_CONSTANT, value=color)
     # overwrite the subsequent channel of original image
            reimage[:,:,n]=sqimage
        
         # save the processed image in the new folder named embedding
        cv2.imwrite("C:/Users/Lenovo/Deep_learning/TanuJ_sir/embedding/"+str(i)+".jpg",reimage)
    

#### find the encoding of all the images in embedded directory and then take average of all of them

In [19]:
# initialize the list to store the encoding of all the images in dataset
z=[]
#run a loop through all the images in embedding folder to find their separate encodings
for i in os.listdir('embedding'):
    arr3=img_to_encoding(os.path.join('embedding',i), FRmodel)
    # append encoding of individual images in list
    z.append(arr3)

In [15]:
#convert the list into numpy array
zr=np.array(z)
#find the sum of all encoding element wise
z_sum=np.sum(zr,axis=0)
#find the average
z_mean=z_sum/count
#z_mean

In [16]:
z_mean.shape

(1, 128)

In [17]:
d=np.linalg.norm((z_mean-arr))
print(d)

0.39778063


In [115]:
#f=dist_image(arr,arr3)
#print(f)